In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
from scipy.interpolate import make_interp_spline

In [3]:
flag_project = 1 # 0--regular ; 1--repartrition

In [4]:
if flag_project == 0:
    file_project = 'a-project.yml'
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server


In [5]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v3.0/output_repar_01-01/


In [6]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [7]:
dir_disp_autopick = dir_project + info_basic['dir_disp']

In [8]:
key_subworks = info_basic['key_subworks']
key_subworks = info_basic['key_subworks'][600:]

In [9]:
nums_repick =[127,137,231,232,366,384,411,412,456,466,467,489,501,502,503,504,507,508,515,516,519,528,530,532,534,544,545,538,546,573,578,579,581,582,589,591,592,593,597,603,604,616,617,634,635,636,637,640,661,662,664,665,667,668]
key_subworks = []
for num in nums_repick:
    tag = str(num)
    key_ds0 = [key_subwork for key_subwork in info_basic['key_subworks'] if tag == key_subwork[-len(str(tag)):]][0]
    key_subworks.append(key_ds0)

### create workspace for Dispernet

In [10]:
info_basic['dir_inv_dispernet'] = 'inv_dispernet/'
dir_inv = dir_project + info_basic['dir_inv_dispernet']
if not os.path.exists(dir_inv):
    os.makedirs(dir_inv)
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [11]:
dir_disp = dir_inv + 'data/'
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)

### Autopick

In [12]:
flag_f = [2,15]

In [13]:
def transfer_autopick(key_subwork):
    global dir_disp_autopick, dir_disp, flag_f
    print(key_subwork)
    ff_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    cc_pick = np.loadtxt(dir_disp_autopick+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    f_start = flag_f[0]
    f_end = flag_f[1]
    ff_this = ff_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    cc_this = cc_pick[(ff_pick>=f_start) & (ff_pick<=f_end)]
    mode = np.zeros(len(ff_this))
    np.savetxt(dir_disp+'ds_'+key_subwork+'curve.txt',np.c_[ff_this,cc_this,mode], fmt='%.6f  %.6f  %i')

In [14]:
for key_subwork in key_subworks:
    #print(key_subwork)
    transfer_autopick(key_subwork)

16-06--127
26-06--137
39-08--231
40-08--232
43-11--366
13-12--384
40-12--411
42-12--412
40-13--456
50-13--466


51-13--467
26-14--489
38-14--501
39-14--502
40-14--503
41-14--504
44-14--507
45-14--508
53-14--515
54-14--516
11-15--519
20-15--528
22-15--530
24-15--532
26-15--534
38-15--544
39-15--545
31-15--538
40-15--546
22-16--573
27-16--578
28-16--579
30-16--581
31-16--582
38-16--589
40-16--591
41-16--592
42-16--593
46-16--597
52-16--603
53-16--604
19-17--616
20-17--617
38-17--634
40-17--635
41-17--636
42-17--637
45-17--640
21-18--661
22-18--662
28-18--664
29-18--665
34-18--667
35-18--668
